In [1]:
from datetime import datetime, timedelta
import math

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xgboost as xgb
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.preprocessing import MinMaxScaler

pd.set_option('display.max_columns', 500)

### Separate features and target variables
    - additionaly we define the probability of a non-good target hit stop loss and a good target to hit stop loss are calulated

In [2]:
def get_x_y(target_value, features_df):
    targets = ['target_log_return_1', 'target_log_return_2', 'target_log_return_3',
                'target_log_return_4', 'target_log_return_5', 'target_log_return_6',]

    mask = (
       ((features_df['target_log_return_1'] >= target_value)
        | (features_df['target_log_return_2'] >= target_value) 
        | (features_df['target_log_return_3'] >= target_value) 
        | (features_df['target_log_return_4'] >= target_value)
        | (features_df['target_log_return_5'] >= target_value)
        | (features_df['target_log_return_6'] >= target_value))
           )
    print('Setting up target variable...')
    features_df.loc[mask, 'target_sign'] = 1
    features_df.loc[~mask, 'target_sign'] = 0
    
    drop_columns = ['coin', 'date', 'period', 
                'high', 'low', 'open', 'close', 'volume', 'quote_volume',
                'weightedAverage',
                'BBUpper', 'BBLower', 'BBMiddle',
                'EMA9', 'EMA12', 'EMA26', 'EMA50',
                'SMA9', 'SMA12', 'SMA26', 'SMA50',
                'target_log_return_1', 'target_log_return_2', 'target_log_return_3',
                'target_log_return_4', 'target_log_return_5', 'target_log_return_6'
               ]

    # Preparing dataframe
    print('Cleaning data set...')
    mask = (
       ((features_df['log_return'] >= target_value)
        | (features_df['log_return_2'] >= target_value) 
        | (features_df['log_return_3'] >= target_value) 
        | (features_df['log_return_4'] >= target_value)
           ))
    features_df.loc[mask, 'last_target'] = 1
    features_df.loc[~mask, 'last_target'] = 0
    clean_df = features_df.drop(drop_columns, axis=1).copy()
    # Defining Features (X) and Target Variable (y)
    print('Getting list of returns...')
    
    returns = []
    for t in targets:
        returns.append(features_df.loc[features_df['target_sign'] == 0, t].values)

    flat_missed_list = [item for sublist in returns for item in sublist if not math.isnan(item)]

    returns = []
    for t in targets:
        returns.append(features_df.loc[features_df['target_sign'] == 1, t].values)

    flat_list = [item for sublist in returns for item in sublist if not math.isnan(item)]
    
    
    X = clean_df.drop(['target_sign'], axis =1).copy()
    y = clean_df[['target_sign']].copy()
    
    return X, y, flat_missed_list, flat_list
    
    

### Running model

In [3]:
def run_model(X_sample, y_sample, target_value, threshold,  model, scale=True, smote=True):
    
        
    if scale:
        print('Scaling...')
        scaler = MinMaxScaler().fit(X_sample)
        X_sample = scaler.transform(X_sample)
    # Split Train, Test
    X_train, X_test = X_sample[:math.floor(len(X_sample) * 0.75)].copy(), X_sample[math.ceil(len(X_sample) * 0.75):].copy()
    y_train, y_test = y_sample.iloc[:math.floor(len(y_sample) * 0.75)].copy(), y_sample.iloc[math.ceil(len(y_sample) * 0.75):].copy()

    if smote:
        print('Balacing dataset...')
        # Rebalance Samples
        oversampler = SMOTE(random_state=42)
        X_train, y_train = oversampler.fit_sample(X_train, y_train.target_sign)
        
    # (Re) define Models
    print('Fitting model...')
    model.fit(X_train, y_train)
    predicted_proba = model.predict_proba(X_test)
    predicted = (predicted_proba[:, 1] >= threshold).astype('int')
    precision = precision_score(y_test, predicted)
    train_acc = accuracy_score(y_test, predicted)
    return {
        'Precision': precision,
        'Accuracy': train_acc,
        'Trades': np.sum(predicted),
        'Real Profits': np.sum(y_test)[0],
        'Target': target_value,
        'Scaler': scaler,
        'Model': model
    }

### For a given stop loss-take profit value, we define de Expected Return of the model:
    - Sum of the given values:
        - Precision of the model is right * Probability of not hitting stop loss mark * Take Profit
        - Precision of the model is right * Probability of hitting stop loss mark * Stop Loss
        - Precision of the model is wrong * Probability of not hitting stop loss mark * Mean Return of Sample
        - Precision of the model is wrong * Probability of hitting stop loss mark * Stop Loss

In [15]:
import pickle
# ['USDT_BTC', 'USDT_ETH', 'USDT_LTC', 'USDT_XRP', 'USDT_ETC', 'USDT_DASH',
#                 'USDT_XMR',  'USDT_STR',]

# Done: 
coin = 'USDT_EOS'
df = pickle.load(open('../' + coin + 'back_test_sample.sav', 'rb'))
df = df.loc[df.date >= datetime(2018,2,1,0,0).timestamp()].sort_values(['date'], ascending=True)
last_row = 0
for index, row in df.dropna().iterrows():
    ok = True
    if last_row == 0:
        last_row = row['date']
        continue
    if row['date'] - last_row  != 14400:
        print('Not consistent')
        print(row.date)
        print(last_row)
    #df.loc[df.index==index, 'dif'] = row['date'] - last_row 
    last_row = row['date']
#df[['date', 'dif']]
date_initial = df.date.iloc[0]
date_end = df.date.iloc[-1]

# plt.hist(df.target_log_return_6, bins=15)
# plt.hist(df.target_log_return_5, bins=15)
# plt.hist(df.target_log_return_4, bins=15)
# plt.hist(df.target_log_return_3, bins=15)
# plt.hist(df.target_log_return_2, bins=15)
# plt.hist(df.target_log_return_1, bins=15)
# plt.show()

### For a given Size of Sample to use in model each step we do the following:
     - Run the model (if enough sample)
     - Select best parameters (Stop Loss and Take Profit)
     - Predict (if Model is good enough)
    

In [16]:
df.columns

Index(['coin', 'date', 'period', 'high', 'low', 'close', 'open', 'volume',
       'quote_volume', 'weightedAverage', 'ADX', 'ATR', 'BBLower', 'BBMiddle',
       'BBUpper', 'EMA12', 'EMA26', 'EMA50', 'EMA9', 'HISTOGRAM', 'MACD',
       'MOMENTUM', 'OBV', 'RSI14', 'SIGNAL', 'SMA12', 'SMA26', 'SMA50', 'SMA9',
       'TREND_COIN', 'WILLR', 'slope', 'slope_short', 'BBand_height',
       'BBand_lower_height', 'BBand_upper_height', 'EMA_height12',
       'EMA_height26', 'SMA_height12', 'SMA_height26', 'close_open',
       'close_low', 'close_high', 'log_trend', 'rate_slope', 'high_low',
       'ema_height', 'log_return', 'log_return_2', 'log_return_3',
       'log_return_4', 'mean_return', 'variance', 'stdev',
       'target_log_return_1', 'target_log_return_2', 'target_log_return_3',
       'target_log_return_4', 'target_log_return_5', 'target_log_return_6'],
      dtype='object')

In [17]:
# import talib as tb
# ma = tb.MA(back_test.TREND_COIN, timeperiod=6)
# back_test['SMA24_trend'] = ma
df

,coin,date,period,high,low,close,open,volume,quote_volume,weightedAverage,ADX,ATR,BBLower,BBMiddle,BBUpper,EMA12,EMA26,EMA50,EMA9,HISTOGRAM,MACD,MOMENTUM,OBV,RSI14,SIGNAL,SMA12,SMA26,SMA50,SMA9,TREND_COIN,WILLR,slope,slope_short,BBand_height,BBand_lower_height,BBand_upper_height,EMA_height12,EMA_height26,SMA_height12,SMA_height26,close_open,close_low,close_high,log_trend,rate_slope,high_low,ema_height,log_return,log_return_2,log_return_3,log_return_4,mean_return,variance,stdev,target_log_return_1,target_log_return_2,target_log_return_3,target_log_return_4,target_log_return_5,target_log_return_6
24322,USDT_EOS,1538064000,14400,5.900000,5.528993,5.900000,5.559899,79535.802011,14020.008675,5.673021,18.898613,0.214497,5.380357,5.647323,5.914288,5.551815,5.590146,5.645723,5.608564,0.041665,0.007924,0.300000,-6.151260e+05,60.398738,-0.033741,5.551815,5.590146,5.645723,5.608564,12745.0,0.000000,-0.018218,-0.016760,1.099237,0.911925,1.002422,0.940986,0.947482,0.940986,0.947482,0.942356,0.937118,1.000000,-0.007270,1.087005,1.067102,1.0,0.059372,0.050362,0.066583,0.043910,0.007959,0.000610,0.024689,-0.008803,-0.014743,0.016807,-0.016013,-0.008514,-0.015466
24333,USDT_EOS,1538078400,14400,5.980000,5.769531,5.848290,5.900000,83250.517499,14191.446479,5.866246,18.727474,0.214210,5.376116,5.687673,5.999230,5.586570,5.587003,5.659955,5.655874,0.048888,0.027369,0.288290,-6.983765e+05,58.378616,-0.021519,5.586570,5.587003,5.659955,5.655874,12629.0,-13.616177,-0.018218,-0.013471,1.115904,0.919263,1.025809,0.955248,0.955322,0.955248,0.955322,1.008842,0.986533,1.022521,-0.009143,1.352376,1.036479,1.0,-0.008803,0.050569,0.041559,0.057780,0.006165,0.000630,0.025090,-0.005940,0.025610,-0.007210,0.000289,-0.006663,-0.013879
24340,USDT_EOS,1538092800,14400,5.864523,5.781831,5.813655,5.848290,36098.980041,6222.684199,5.801191,18.568559,0.204816,5.475379,5.746413,6.017448,5.628061,5.581540,5.670730,5.673173,0.048838,0.039528,0.167117,-7.344755e+05,57.003392,-0.009310,5.628061,5.581540,5.670730,5.673173,12521.0,-22.402266,-0.013543,-0.010372,1.099001,0.941813,1.035054,0.968076,0.960074,0.968076,0.960074,1.005957,0.994526,1.008750,-0.008589,1.305739,1.014302,1.0,-0.005940,-0.014743,0.044629,0.035619,0.007461,0.000571,0.023905,0.031550,-0.001270,0.006229,-0.000723,-0.007940,-0.037443
24363,USDT_EOS,1538107200,14400,6.029000,5.753608,6.000000,5.773190,144463.029271,24563.162813,5.881288,19.174020,0.209857,5.531525,5.824369,6.117213,5.678252,5.578301,5.686942,5.717618,0.058224,0.063470,0.480045,-5.900125e+05,62.167283,0.005246,5.678252,5.578301,5.686942,5.717618,12425.0,-3.663770,-0.013543,-0.005639,1.105882,0.921921,1.019536,0.946375,0.929717,0.946375,0.929717,0.962198,0.958935,1.004833,-0.007697,2.401729,1.047864,1.0,0.031550,0.025610,0.016807,0.076179,0.008815,0.000617,0.024832,-0.032820,-0.025321,-0.032273,-0.039490,-0.068993,-0.069110
24367,USDT_EOS,1538121600,14400,6.018151,5.754779,5.806275,6.018151,110243.529936,18890.903543,5.835800,19.736234,0.213679,5.730982,5.873644,6.016306,5.710233,5.570783,5.698271,5.744982,0.048643,0.066050,0.196053,-7.002560e+05,54.798941,0.017407,5.710233,5.570783,5.698271,5.744982,12366.0,-28.138449,-0.013543,-0.001375,1.049786,0.987033,1.036173,0.983459,0.959442,0.983459,0.959442,1.036491,0.991131,1.036491,-0.004760,9.849055,1.045766,1.0,-0.032820,-0.001270,-0.007210,-0.016013,0.005699,0.000762,0.027604,0.007499,0.000547,-0.006669,-0.036173,-0.036290,-0.004803
24380,USDT_EOS,1538136000,14400,5.861778,5.688722,5.849981,5.820250,80623.193815,13945.009901,5.781509,19.754253,0.210778,5.722770,5.863640,6.004510,5.726235,5.574417,5.710271,5.767586,0.042719,0.070806,0.290081,-6.196328e+05,56.064177,0.028087,5.726235,5.574417,5.710271,5.767586,12261.0,-25.249491,-0.013543,0.002152,1.049231,0.978254,1.026415,0.978847,0.952895,0.978847,0.952895,0.994918,0.972434,1.002017,-0.008527,-6.292118,1.030421,1.0,0.007499,-0.025321,0.006229,0.000289,0.002781,0.000630,0.025096,-0.006952,-0.014169,-0.043672,-0.043789,-0.012302,-0.015983
24385,

In [9]:
threshold = 0.50
risk_free = ((1 + 0.065) ** (1/252) - 1)
test_size = -48


models = {
    'XGBoost': xgb.XGBClassifier(max_depth=8, learning_rate=0.2, n_estimators=400,
                        objective= "binary:logistic", eval_metric="error", gamma=0.,  #map
                        max_delta_step=3, subsample=0.9, colsample_bytree=0.4,
                        seed=0, )  
}



precisions = []
for target_value in [0.005, 0.0075, 0.01, 0.0125, 0.015, 0.02, 0.0225, 0.025, 0.0275, 0.03, 0.0325]:
    print('Initiating ....')
    back_test = df.copy().sort_values('date')
    
    print('Getting X Y Variables....')
    print(len(back_test))
    X, y, flat_missed_list, flat_list = get_x_y(target_value, back_test.iloc[:test_size])
    
    print('Running model ....')
    model = run_model(X, y, target_value, threshold, models['XGBoost'], scale=True, smote=True)
    print(model['Precision'])
    if model['Precision'] > 0.5 and np.percentile(flat_missed_list, 30) < 0:
        precisions.append({
            'target_value': target_value,
            'precision': model['Precision'],
            'model': model,
            'stop_loss': np.percentile(flat_missed_list, 25)
        })

Initiating ....
Getting X Y Variables....
20
Setting up target variable...


ValueError: cannot set a frame with no defined index and a scalar

In [ ]:
df_models = pd.DataFrame.from_records(precisions)

In [ ]:
df_models

In [ ]:
plt.plot(df_models['target_value'], df_models['precision'])
plt.show()

In [ ]:
if df_models.empty:
    print('No Model worth fighting for... ')
possible_models = df_models.sort_values('target_value', ascending=False)

model = possible_models.iloc[0]
target_value = model['model']['Target']
stop_loss = model['stop_loss']
model = model['model']

In [ ]:
risk_free = ((1 + 0.065) ** (1/252) - 1)

drop_columns = ['coin', 'date', 'period', 
                'high', 'low', 'open', 'close', 'volume', 'quote_volume',
                'weightedAverage',
                'BBUpper', 'BBLower', 'BBMiddle',
                'EMA9', 'EMA12', 'EMA26', 'EMA50',
                'SMA9', 'SMA12', 'SMA26', 'SMA50',
                'target_log_return_1', 'target_log_return_2', 'target_log_return_3',
                'target_log_return_4', 'target_log_return_5', 'target_log_return_6'
               ]


back_test = df.copy()
mask = (
       ((back_test['log_return'] >= target_value)
        | (back_test['log_return_2'] >= target_value) 
        | (back_test['log_return_3'] >= target_value) 
        | (back_test['log_return_4'] >= target_value)
           ))
back_test.loc[mask, 'last_target'] = 1
back_test.loc[~mask, 'last_target'] = 0
for index, row in back_test.iloc[-test_size:].iterrows():
    signal = 0
    current_date = row['date']
    X = row.drop(drop_columns)
    X = model['Scaler'].transform(X.ravel().reshape(1, -1))
    predicted_proba = model['Model'].predict_proba(X)
    if predicted_proba[0][1] >= threshold:
        signal = 1
    back_test.loc[back_test['date'] == row['date'], 'predicted_signal'] = signal

In [ ]:
mask_profit = (
       ((back_test['target_log_return_1'] >= target_value)
        | (back_test['target_log_return_2'] >= target_value) 
        | (back_test['target_log_return_3'] >= target_value) 
        | (back_test['target_log_return_4'] >= target_value)
        | (back_test['target_log_return_5'] >= target_value)
        | (back_test['target_log_return_6'] >= target_value))
           )

mask_loss =  (
       ((back_test['target_log_return_1'] <= stop_loss)
        | (back_test['target_log_return_2'] <= stop_loss) 
        | (back_test['target_log_return_3'] <= stop_loss) 
        | (back_test['target_log_return_4'] <= stop_loss)
        | (back_test['target_log_return_5'] <= stop_loss)
        | (back_test['target_log_return_6'] <= stop_loss))
           )


back_test.loc[mask_profit & (~mask_loss), 'target_sign'] = 1
back_test.loc[mask_loss, 'target_sign'] = 0

In [ ]:
for index, row in back_test.iterrows():
    hit = [row.target_log_return_1,
             row.target_log_return_2,
             row.target_log_return_3,
            row.target_log_return_4,
            row.target_log_return_5,
            row.target_log_return_6]
    hit = [j for j in hit if j <= stop_loss]
    if hit:
        back_test.loc[back_test.index == index, 'hit'] = max(hit)
    else:
        back_test.loc[back_test.index == index, 'hit'] = 0

In [ ]:
results = back_test.dropna()
results.loc[(results['predicted_signal'] == 1) & (results['target_sign'] == 1), 'profit'] = target_value
results.loc[(results['predicted_signal'] == 1) & (results['target_sign'] == 0), 'profit'] = results.loc[(results['predicted_signal'] == 1) & (results['target_sign'] == 0), 'hit']
results.loc[(results['profit'].isnull()), 'profit'] = 0

In [ ]:
results['acc_return'] = results['profit'].cumsum().copy()

In [ ]:
print('All data to model from:')
print('------------------')
print('Starting data: ' + datetime.fromtimestamp(date_initial).strftime('%Y-%m-%d %H:%M:%S'))
print('------------------')
print('End data: ' + datetime.fromtimestamp(date_end).strftime('%Y-%m-%d %H:%M:%S'))
print('------------------')
print('Stop Loss: ' + str(stop_loss))
print('------------------')
print('Target Value: ' + str(target_value))
print('------------------')
print('Model Precision: ' + str(model['Precision']))
print('------------------')
print('Backtesting data :  ' + str((datetime.fromtimestamp(back_test.iloc[-1].date) - datetime.fromtimestamp(back_test.iloc[-test_size].date)).days ))
print('------------------')
print('Starting data: ' + datetime.fromtimestamp(back_test.iloc[-test_size].date).strftime('%Y-%m-%d %H:%M:%S'))
print('------------------')
print('End data: ' + datetime.fromtimestamp(back_test.iloc[-1].date).strftime('%Y-%m-%d %H:%M:%S'))
print('------------------')
print('Acc. Return Test: ' + str(results['acc_return'].iloc[-1]))

In [ ]:
plt.plot(results.date, results.acc_return)
plt.xticks(rotation=75)
plt.show()

In [ ]:
plt.hist(results['profit'], bins=30)
plt.show()

In [ ]:
results[['predicted_signal', 'target_sign', 'profit', 'acc_return']].tail()

In [ ]:
from sklearn.feature_selection import SelectFromModel

selection = SelectFromModel(model['Model'], threshold=0.03, prefit=True) 

In [ ]:
plt.bar(range(len(model['Model'].feature_importances_)), model['Model'].feature_importances_)
plt.show()

In [ ]:
plt.hist(y, bins=3)
plt.show()

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

xgb_model = models['XGBoost']
    
back_test = df.copy().sort_values('date')
mask = (
       ((back_test['log_return'] >= target_value)
        | (back_test['log_return_2'] >= target_value) 
        | (back_test['log_return_3'] >= target_value) 
        | (back_test['log_return_4'] >= target_value)
           ))
back_test.loc[mask, 'last_target'] = 1
back_test.loc[~mask, 'last_target'] = 0
X, y, flat_missed_list, flat_list = get_x_y(target_value, back_test.iloc[:])
# X = selection.transform(X)
y_sample = y
X_sample = X
scaler = MinMaxScaler().fit(X_sample)
X_sample = scaler.transform(X_sample)
# Split Train, Test
X_train, X_test = X_sample[:math.floor(len(X_sample) * 0.75)], X_sample[math.ceil(len(X_sample) * 0.75):]
y_train, y_test = y_sample.iloc[:math.floor(len(y_sample) * 0.75)], y_sample.iloc[math.ceil(len(y_sample) * 0.75):]

# Rebalance Samples
oversampler = SMOTE(random_state=42)
X_train, y_train = oversampler.fit_sample(X_train, y_train.target_sign)

n_estimators = [100, 200, 300, 400, 500]
max_depth = [2, 4, 6, 8]
max_delta_step = [1, 2, 3, 4, 5]
param_grid = dict(max_depth=max_depth, n_estimators=n_estimators,max_delta_step=max_delta_step)
kfold = StratifiedKFold(n_splits=2, shuffle=False, random_state=7)
grid_search = GridSearchCV(xgb_model, param_grid, scoring="precision", n_jobs=-1, cv=kfold, verbose=1)
grid_result = grid_search.fit(X_train, y_train)

In [ ]:
selected_model = xgb.XGBClassifier(max_depth=grid_result.best_params_['max_depth'],
                                 learning_rate=0.2, n_estimators=grid_result.best_params_['n_estimators'],
                                 objective= "binary:logistic", eval_metric="error", gamma=0.,  #map
                                 max_delta_step=grid_result.best_params_['max_delta_step'], subsample=0.9,
                                 colsample_bytree=0.4,
                    seed=0, )  
selected_model.fit(X_train, y_train)

predicted_proba = selected_model.predict_proba(X_test)
predicted = (predicted_proba[:, 1] >= threshold).astype('int')
precision = precision_score(y_test, predicted)
train_acc = accuracy_score(y_test, predicted)

In [ ]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

In [ ]:
precision

In [ ]:
target_value

In [ ]:
stop_loss

In [ ]:
print(str(abs(target_value/stop_loss)))

In [ ]:
df_models

In [ ]:
plt.bar(range(len(xgb_model.feature_importances_)), xgb_model.feature_importances_)
plt.show()

In [ ]:
model = {
    'take_profit': target_value,
    'stop_loss': stop_loss,
    'Scaler': X_sample,
    'Model': selected_model
    
}
filename = '../' + coin + '_model.sav'
pickle.dump(model, open(filename, 'wb'))